In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train = pd.read_csv("train.csv")
train['cfips'] = train['cfips'].apply(str)
train['date'] = train['first_day_of_month'].apply(lambda s: pd.to_datetime(s))
train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019-08-01
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019-09-01
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019-10-01
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019-11-01
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019-12-01
...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022-06-01
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022-07-01
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022-08-01
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022-09-01


In [ ]:
train[['microbusiness_density', 'active']].corr(method='spearman')

In [ ]:
train[['microbusiness_density', 'active']].describe()

In [4]:
cfips_list = list(set(train['cfips']))

## Detect and clean outliers
Using the IQR method. If an obs is under $IQR - 1.5*IQR$ or above $IQR + 1.5*IQR$ it is an outlier. 

If it is an outlier, replace with the average of the previous and next obs. If the prev obs is not available (i.e. it is the first obs) use the next; if the next obs is not available (i.e. it is the last obs) use the prev.

In [119]:


res = []
outlier_indices = {}
for cfip in cfips_list:
    X = train.query(f"cfips == '{cfip}'").sort_values('date', ascending=True)
    
    y = X['microbusiness_density']
    q025, q75 = np.quantile(y, q = [0.25, 0.75])
    lo_iqr, hi_iqr = q025 - 1.5*q025, q75 + 1.5*q75
    
    def outlier(x, lo_iqr, hi_iqr):
        if x < 2*lo_iqr:
            return True
        if x > 2*hi_iqr:
            return True
        return False

    X['outlier'] = y.apply(outlier, args=(lo_iqr, hi_iqr))
    
    idx = (X['outlier'] == True)
    if sum(idx) > 0:
        print(cfip)
        outlier_indices[cfip] = idx
        X.loc[idx, ['microbusiness_density', 'active']] = np.nan

        X['microbusiness_density_ffill'] = X['microbusiness_density'].fillna(method='ffill').fillna(method='bfill')
        X['microbusiness_density_bfill'] = X['microbusiness_density'].fillna(method='bfill').fillna(method='ffill')
        X['microbusiness_density'] = .5*(X['microbusiness_density_bfill'] + X['microbusiness_density_ffill'])
        
        X['active_ffill'] = X['active'].fillna(method='ffill').fillna(method='bfill')
        X['active_bfill'] = X['active'].fillna(method='bfill').fillna(method='ffill')
        X['active'] = .5*(X['active_bfill'] + X['active_ffill'])
        
        X = X.drop(labels=['microbusiness_density_ffill', 
                       'microbusiness_density_bfill',
                       'active_ffill',
                       'active_bfill'
                      ],
               axis=1
              )

    res.append(X)
train_imputed = pd.concat(res, axis=0)

print(train_imputed.shape)
train_imputed.query("outlier == True")

46127
31009
29171
29063
(122265, 9)


,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date,outlier
94380,46127_2019-08-01,46127,Union County,South Dakota,2019-08-01,5.880263,662.0,2019-08-01,True
94382,46127_2019-10-01,46127,Union County,South Dakota,2019-10-01,5.875822,661.5,2019-10-01,True
64595,31009_2020-07-01,31009,Blaine County,Nebraska,2020-07-01,0.266667,1.0,2020-07-01,True
61083,29171_2020-05-01,29171,Putnam County,Missouri,2020-05-01,1.437399,53.5,2020-05-01,True
59006,29063_2022-10-01,29063,DeKalb County,Missouri,2022-10-01,1.323290,130.0,2022-10-01,True


In [120]:
train_imputed

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date,outlier
117624,54063_2019-08-01,54063,Monroe County,West Virginia,2019-08-01,0.859202,93.0,2019-08-01,False
117625,54063_2019-09-01,54063,Monroe County,West Virginia,2019-09-01,0.877679,95.0,2019-09-01,False
117626,54063_2019-10-01,54063,Monroe County,West Virginia,2019-10-01,0.914634,99.0,2019-10-01,False
117627,54063_2019-11-01,54063,Monroe County,West Virginia,2019-11-01,0.933112,101.0,2019-11-01,False
117628,54063_2019-12-01,54063,Monroe County,West Virginia,2019-12-01,0.914634,99.0,2019-12-01,False
...,...,...,...,...,...,...,...,...,...
94960,47021_2022-06-01,47021,Cheatham County,Tennessee,2022-06-01,5.811858,1838.0,2022-06-01,False
94961,47021_2022-07-01,47021,Cheatham County,Tennessee,2022-07-01,5.856126,1852.0,2022-07-01,False
94962,47021_2022-08-01,47021,Cheatham County,Tennessee,2022-08-01,5.878261,1859.0,2022-08-01,False
94963,47021_2022-09-01,47021,Cheatham County,Tennessee,2022-09-01,5.897233,1865.0,2022-09-01,False


In [121]:
train_imputed.query("outlier == True")

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date,outlier
94380,46127_2019-08-01,46127,Union County,South Dakota,2019-08-01,5.880263,662.0,2019-08-01,True
94382,46127_2019-10-01,46127,Union County,South Dakota,2019-10-01,5.875822,661.5,2019-10-01,True
64595,31009_2020-07-01,31009,Blaine County,Nebraska,2020-07-01,0.266667,1.0,2020-07-01,True
61083,29171_2020-05-01,29171,Putnam County,Missouri,2020-05-01,1.437399,53.5,2020-05-01,True
59006,29063_2022-10-01,29063,DeKalb County,Missouri,2022-10-01,1.323290,130.0,2022-10-01,True


In [112]:
train_imputed.drop(labels=['outlier'], axis=1).sort_values(['cfips', 'date'], ascending=True).to_csv("train-imputed.csv", index=False)

In [127]:
for cfip in cfips_list:
    X = train_imputed.query(f"cfips == '{cfip}'").sort_values('date', ascending=True)
    y = X['microbusiness_density']
    q05, q025, q75, q95 = np.quantile(y, q = [0.05, 0.25, 0.75, .95])
    
    lo_iqr, hi_iqr = q025 - 1.5*q025, q75 + 1.5*q75
    lo_iqr, hi_iqr = q05, q95
    med = np.median(y)
    if (min(y) < lo_iqr) or (max(y) > hi_iqr):
        print(cfip, '-- loiqr:', lo_iqr, 'hiiqr:', hi_iqr, 'min:', min(y), 'median:', med, 'max:', max(y))

54063 -- loiqr: 0.9046615 hiiqr: 1.1971047 min: 0.85920179 median: 1.0686164 max: 1.2688709
29227 -- loiqr: 1.23334966 hiiqr: 1.4715099999999999 min: 1.1385199 median: 1.3605442 max: 1.546073
39071 -- loiqr: 1.6106846700000002 hiiqr: 1.85214619 min: 1.5959399 median: 1.7470045 max: 1.8741592
6069 -- loiqr: 4.26980847 hiiqr: 5.00951786 min: 4.2635746 median: 4.7049084 max: 5.0346804
35059 -- loiqr: 1.2500000599999999 hiiqr: 1.48005911 min: 1.2224939 median: 1.3648771 max: 1.506647
29019 -- loiqr: 6.6027743899999995 hiiqr: 7.1067889300000004 min: 6.4339795 median: 6.8513999 max: 7.1487088
29149 -- loiqr: 1.5884847800000002 hiiqr: 2.05184654 min: 1.5317286 median: 1.645448 max: 2.1521153
1051 -- loiqr: 2.6402397900000003 hiiqr: 3.35869308 min: 2.5783718 median: 2.7656362 max: 3.4046247
34023 -- loiqr: 7.43299673 hiiqr: 7.90512219 min: 7.4144273 median: 7.752809 max: 7.935492
40001 -- loiqr: 0.7312277 hiiqr: 0.994137593 min: 0.7312277 median: 0.9617756 max: 1.0049322
49013 -- loiqr: 4.0827

6045 -- loiqr: 5.97603694 hiiqr: 6.50235752 min: 5.8944573 median: 6.2234612 max: 6.5690765
42087 -- loiqr: 2.17374897 hiiqr: 2.3083928499999997 min: 2.1607773 median: 2.2359591 max: 2.3099186
1067 -- loiqr: 2.42408598 hiiqr: 2.7531799 min: 2.4027123 median: 2.6446037 max: 2.8235643
13263 -- loiqr: 0.64638782 hiiqr: 1.0128033 min: 0.62405449 median: 0.84081787 max: 1.0701319
20015 -- loiqr: 3.97014572 hiiqr: 4.09804176 min: 3.9412861 median: 3.9985476 max: 4.115499
50011 -- loiqr: 3.69895887 hiiqr: 5.218952209999999 min: 3.6814778 median: 4.0543017 max: 5.2329597
12089 -- loiqr: 6.25519589 hiiqr: 6.76725517 min: 6.1392565 median: 6.3932428 max: 6.8235364
27023 -- loiqr: 1.71345466 hiiqr: 2.17286384 min: 1.6917905 median: 1.9751201 max: 2.1827354
48335 -- loiqr: 0.884000643 hiiqr: 1.0707241 min: 0.84857351 median: 0.95098758 max: 1.1000144
17143 -- loiqr: 4.1005016 hiiqr: 4.53508261 min: 4.0654163 median: 4.3069963 max: 4.5723891
55115 -- loiqr: 1.88528686 hiiqr: 1.97073692 min: 1.87344

32019 -- loiqr: 3.19375285 hiiqr: 3.66754866 min: 3.1820564 median: 3.3484535 max: 3.7118113
55083 -- loiqr: 2.87412422 hiiqr: 3.9181596200000004 min: 2.7638526 median: 3.5859358 max: 3.969723
56005 -- loiqr: 2.4135726 hiiqr: 2.76389862 min: 2.3833656 median: 2.6624444 max: 2.7973258
38057 -- loiqr: 1.63174319 hiiqr: 2.3519663100000003 min: 1.6049933 median: 1.7755134 max: 2.3965807
54049 -- loiqr: 1.88153 hiiqr: 2.12532727 min: 1.8638215 median: 2.056983 max: 2.1457417
8059 -- loiqr: 10.417515700000001 hiiqr: 11.405503 min: 10.379384 median: 10.705688 max: 11.479008
21157 -- loiqr: 2.80791672 hiiqr: 2.9709737 min: 2.7916734 median: 2.8890772 max: 2.9830346
17141 -- loiqr: 2.7385893 hiiqr: 3.3755624600000003 min: 2.7360744 median: 3.2357786 max: 3.3925686
51115 -- loiqr: 5.17269163 hiiqr: 5.96009645 min: 5.1605968 median: 5.4436293 max: 5.9855385
4021 -- loiqr: 4.3149457600000005 hiiqr: 4.66213764 min: 4.3044987 median: 4.5041318 max: 4.6939907
29151 -- loiqr: 1.6792578599999999 hiiqr:

27173 -- loiqr: 1.8818124699999998 hiiqr: 2.09132059 min: 1.8593689 median: 1.9462465 max: 2.1166134
54037 -- loiqr: 5.319610160000001 hiiqr: 6.0615156500000005 min: 5.2671218 median: 5.6545401 max: 6.088737
33003 -- loiqr: 7.71397847 hiiqr: 8.11818274 min: 7.6795268 median: 7.9478383 max: 8.1849365
6043 -- loiqr: 7.1111112400000005 hiiqr: 7.871647100000001 min: 7.0919065 median: 7.3022451 max: 7.9698215
40133 -- loiqr: 1.0093759900000001 hiiqr: 1.13891073 min: 0.99311227 median: 1.1105772 max: 1.1549826
45019 -- loiqr: 12.1089655 hiiqr: 13.8054285 min: 12.007542 median: 12.603252 max: 14.086524
28155 -- loiqr: 0.618015381 hiiqr: 0.830324915 min: 0.60589743 median: 0.73815596 max: 0.92394215
19135 -- loiqr: 1.0390481 hiiqr: 1.20500102 min: 1.0390481 median: 1.1369421 max: 1.2217885
28139 -- loiqr: 1.30382269 hiiqr: 1.57619478 min: 1.2681925 median: 1.4186538 max: 1.5930268
48237 -- loiqr: 1.8944189 hiiqr: 2.2888825 min: 1.8491973 median: 2.1072247 max: 2.3173158
39009 -- loiqr: 4.22722

45075 -- loiqr: 1.71470889 hiiqr: 2.1336950800000003 min: 1.6968768 median: 2.0156152 max: 2.1658361
9015 -- loiqr: 3.52176386 hiiqr: 4.04863523 min: 3.4920397 median: 3.5848753 max: 4.0765867
17117 -- loiqr: 2.09269696 hiiqr: 2.19723236 min: 2.0901721 median: 2.1492455 max: 2.3184226
12119 -- loiqr: 2.98367329 hiiqr: 3.3524501100000004 min: 2.9006572 median: 3.2501907 max: 3.3622091
50023 -- loiqr: 6.59072032 hiiqr: 6.75996643 min: 6.5599799 median: 6.6514344 max: 6.8234549
37185 -- loiqr: 1.15408152 hiiqr: 1.5479723900000002 min: 1.1258767 median: 1.4519503 max: 1.5764095
5013 -- loiqr: 0.28409091 hiiqr: 0.573505558 min: 0.28409091 median: 0.37878788 max: 0.64485312
41061 -- loiqr: 50.9558206 hiiqr: 61.1653023 min: 50.874134 median: 53.367092 max: 61.442341
29039 -- loiqr: 2.42836945 hiiqr: 2.84982761 min: 2.1175578 median: 2.7372944 max: 2.8961136
42077 -- loiqr: 5.40514977 hiiqr: 6.13468521 min: 5.3814487 median: 5.6268535 max: 6.170928
48047 -- loiqr: 0.37357814899999997 hiiqr: 0.

...

5119 -- loiqr: 5.93299163 hiiqr: 6.19339742 min: 5.881084 median: 6.0312605 max: 6.2127175
48207 -- loiqr: 0.57557023 hiiqr: 0.83780879 min: 0.49710023 median: 0.68463844 max: 0.83780879
41057 -- loiqr: 4.14069201 hiiqr: 4.538717279999999 min: 4.1018915 median: 4.3490605 max: 4.5904121
26075 -- loiqr: 3.0386727099999997 hiiqr: 3.23577494 min: 3.0276442 median: 3.1290753 max: 3.2778041
1055 -- loiqr: 2.24701831 hiiqr: 2.7280282 min: 2.2291651 median: 2.3666215 max: 2.7364635
17053 -- loiqr: 2.05695898 hiiqr: 2.23182714 min: 2.0384278 median: 2.1305311 max: 2.3040586
47013 -- loiqr: 1.2953777 hiiqr: 1.56843563 min: 1.2858993 median: 1.4315577 max: 1.5775857
21203 -- loiqr: 0.738987687 hiiqr: 0.826086983 min: 0.71761203 median: 0.78565979 max: 0.84668189
26149 -- loiqr: 2.5328387 hiiqr: 2.8288335 min: 2.5245473 median: 2.6630256 max: 2.8969324
6019 -- loiqr: 3.69901323 hiiqr: 3.97008813 min: 3.6515334 median: 3.766459 max: 3.9791877
29071 -- loiqr: 3.7095653000000004 hiiqr: 3.99486605 min

18107 -- loiqr: 2.0164177800000003 hiiqr: 2.22957381 min: 2.0133774 median: 2.086699 max: 2.366924
42089 -- loiqr: 4.898294389999999 hiiqr: 5.0874516299999994 min: 4.8199162 median: 4.9488854 max: 5.2292309
24017 -- loiqr: 5.6973449 hiiqr: 6.37158608 min: 5.5990415 median: 6.1647282 max: 6.4074459
17187 -- loiqr: 1.29156347 hiiqr: 1.59040263 min: 1.2845407 median: 1.4488424 max: 1.6521611
18141 -- loiqr: 4.31076571 hiiqr: 4.74410065 min: 4.2820077 median: 4.4591022 max: 4.7759824
36011 -- loiqr: 2.64863486 hiiqr: 2.98163584 min: 2.6293304 median: 2.6935647 max: 3.0012372
48241 -- loiqr: 1.9572474199999998 hiiqr: 2.17988689 min: 1.8701338 median: 2.1217301 max: 2.1866646
20161 -- loiqr: 5.37417046 hiiqr: 5.6067881 min: 5.2346425 median: 5.4962955 max: 5.6307073
28115 -- loiqr: 0.960403285 hiiqr: 1.10311948 min: 0.9565255 median: 1.0055964 max: 1.1107801
5117 -- loiqr: 0.6466584409999999 hiiqr: 0.939539392 min: 0.61881191 median: 0.74866313 max: 0.95411175
12083 -- loiqr: 5.0280781600000

51045 -- loiqr: 1.55861863 hiiqr: 2.05140405 min: 1.5133601 median: 1.8291216 max: 2.1656353
13089 -- loiqr: 12.3980462 hiiqr: 13.3782025 min: 12.116005 median: 12.780334 max: 13.395468
45091 -- loiqr: 6.7932491399999995 hiiqr: 7.49636145 min: 6.7438297 median: 6.9595232 max: 7.5103879
2230 -- loiqr: 6.3392859 hiiqr: 8.4474888 min: 6.25 median: 7.5949368 max: 8.4474888
18023 -- loiqr: 1.8820178 hiiqr: 2.10721352 min: 1.8641589 median: 1.9611137 max: 2.1307955
27165 -- loiqr: 1.59865169 hiiqr: 2.06942437 min: 1.504755 median: 1.942802 max: 2.0924044
13309 -- loiqr: 0.66828674 hiiqr: 0.794399645 min: 0.66717207 median: 0.69866341 max: 0.85222948
20141 -- loiqr: 1.3953179 hiiqr: 2.3252445500000003 min: 1.2461059 median: 1.4783527 max: 2.3904383
29145 -- loiqr: 1.56352012 hiiqr: 1.76089993 min: 1.5551324 median: 1.6276751 max: 1.7696921
45035 -- loiqr: 4.92629851 hiiqr: 5.407920600000001 min: 4.8143387 median: 5.1662593 max: 5.6495533
51760 -- loiqr: 7.14763272 hiiqr: 7.87676826 min: 7.075

55067 -- loiqr: 2.04282295 hiiqr: 23.0558081 min: 2.0247104 median: 2.1928973 max: 25.997801
46101 -- loiqr: 1.92750605 hiiqr: 2.5010335 min: 1.7879162 median: 2.4390244 max: 2.5256674
55109 -- loiqr: 5.5726925000000005 hiiqr: 5.70427786 min: 5.5089407 median: 5.6326632 max: 5.7195916
30051 -- loiqr: 1.0149573 hiiqr: 1.32563256 min: 1.0149573 median: 1.210121 max: 1.430143
17093 -- loiqr: 5.57379006 hiiqr: 5.82852858 min: 5.4993477 median: 5.6473269 max: 5.8660054
19109 -- loiqr: 1.4811992200000001 hiiqr: 1.7560975 min: 1.4266419 median: 1.5832263 max: 1.7971759
40027 -- loiqr: 5.24562859 hiiqr: 5.898034920000001 min: 5.2153511 median: 5.3486257 max: 6.0030127
8103 -- loiqr: 2.72702084 hiiqr: 3.4566881 min: 2.7179384 median: 3.2557182 max: 3.4939013
13247 -- loiqr: 8.2061128 hiiqr: 10.357411200000001 min: 8.048399 median: 9.302947 max: 10.466195
34021 -- loiqr: 8.499056340000001 hiiqr: 10.832245599999998 min: 8.3182926 median: 8.6865387 max: 10.838427
21175 -- loiqr: 0.462385061 hiiqr:

27071 -- loiqr: 1.8481705800000001 hiiqr: 2.16485681 min: 1.8229674 median: 1.9399493 max: 2.2013819
1085 -- loiqr: 1.0258016 hiiqr: 1.4472336 min: 1.0118897 median: 1.3319672 max: 1.4509652
37107 -- loiqr: 1.87922574 hiiqr: 2.2071577 min: 1.8551117 median: 2.0284252 max: 2.2229869
37023 -- loiqr: 2.02281851 hiiqr: 2.20344846 min: 2.0177016 median: 2.161154 max: 2.2197967
39105 -- loiqr: 1.25386829 hiiqr: 1.38579465 min: 1.2488948 median: 1.3096817 max: 1.4006838
48415 -- loiqr: 1.02186972 hiiqr: 1.66575037 min: 0.99337751 median: 1.4607712 max: 1.7199403
1111 -- loiqr: 1.44197419 hiiqr: 4.9313540300000005 min: 1.4158185 median: 2.2195933 max: 5.3465791
20125 -- loiqr: 1.9200153400000002 hiiqr: 2.65119682 min: 1.8966203 median: 2.0805612 max: 2.657151
17125 -- loiqr: 1.00167943 hiiqr: 1.2237272 min: 0.95211941 median: 1.0711343 max: 1.2424101
41011 -- loiqr: 2.9950145 hiiqr: 3.40879156 min: 2.9807897 median: 3.3372512 max: 3.4299784
48337 -- loiqr: 3.79077538 hiiqr: 4.35685186 min: 3.5

In [124]:
train_imputed.query("cfips == '12045'")

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,date,outlier
13221,12045_2019-08-01,12045,Gulf County,Florida,2019-08-01,4.408387,595.0,2019-08-01,False
13222,12045_2019-09-01,12045,Gulf County,Florida,2019-09-01,4.489887,606.0,2019-09-01,False
13223,12045_2019-10-01,12045,Gulf County,Florida,2019-10-01,4.497296,607.0,2019-10-01,False
13224,12045_2019-11-01,12045,Gulf County,Florida,2019-11-01,4.519523,610.0,2019-11-01,False
13225,12045_2019-12-01,12045,Gulf County,Florida,2019-12-01,4.519523,610.0,2019-12-01,False
13226,12045_2020-01-01,12045,Gulf County,Florida,2020-01-01,4.552227,611.0,2020-01-01,False
13227,12045_2020-02-01,12045,Gulf County,Florida,2020-02-01,4.500074,604.0,2020-02-01,False
13228,12045_2020-03-01,12045,Gulf County,Florida,2020-03-01,4.418119,593.0,2020-03-01,False
13229,12045_2020-04-01,12045,Gulf County,Florida,2020-04-01,4.522426,607.0,2020-04-01,False
13230,12045_2020-05-01,12045,Gulf County,Florida,2020-05-01,18.693190,2509.0,2020-05-01,False


## EDA

In [ ]:
cfips = "17031"
X = train.query(f"cfips == '{cfips}'")
print(X.shape)
plt.figure(figsize=(16,8))
plt.plot(X['date'], X['microbusiness_density'])
plt.show()

plt.figure(figsize=(16,8))
plt.plot(X['date'], X['active'])
plt.show()

X

In [ ]:
X[['microbusiness_density', 'active']].corr(method='spearman')

In [ ]:
sample_submissions = pd.read_csv("sample_submission.csv")
sample_submissions['cfips'] = sample_submissions['row_id'].apply(lambda s: s.split("_")[0]).apply(str)
sample_submissions['date'] = sample_submissions['row_id'].apply(lambda s: s.split("_")[1]).apply(pd.to_datetime)
sample_submissions = sample_submissions.sort_values("row_id", ascending=True)
sample_submissions

In [ ]:
predictions_date = sorted(list(set(sample_submissions['date'])))
predictions_date

In [ ]:
last_train_date = pd.to_datetime("2022-10-01")
last_train_date

## Naive baseline #1
* take the last observed value and forecast it forward for the entire forecast period

In [ ]:
cfips_list = list(set(train['cfips']))
print(len(cfips_list))
cfips_list[:10]

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [ ]:
predictions_df = pd.DataFrame({'date':predictions_date})
predictions_df

In [ ]:
h = 8
res = []
last_dates = []
for cfip in cfips_list:
    X = train.query(f"cfips == '{cfip}'").sort_values('date', ascending=True)
    last = X.iloc[-1,]
    last_y = last['microbusiness_density']
    #print(cfip, last_y)
    _predictions_df = predictions_df.copy()
    _predictions_df['cfips'] = cfip
    _predictions_df['microbusiness_density'] = [last_y for _ in range(h)]
    res.append(_predictions_df)
    
submissions_df = pd.concat(res, axis=0)
submissions_df

In [ ]:
submissions_df['microbusiness_density'] = submissions_df['microbusiness_density'] + .01

In [ ]:
np.quantile(submissions_df['microbusiness_density'], q=[.01, .05, .5, .95, .99])

In [ ]:
plt.hist(submissions_df['microbusiness_density'])
plt.show()

plt.hist(np.log(submissions_df['microbusiness_density']))
plt.show()

In [ ]:
x = train[['cfips', 'microbusiness_density']].groupby('cfips').mean().reset_index()
plt.hist(x['microbusiness_density'])
plt.show()

plt.hist(np.log(x['microbusiness_density']))
plt.show()

In [ ]:
plt.hist(np.log(submissions_df['microbusiness_density']), color='red')
plt.hist(np.log(x['microbusiness_density']))


In [ ]:
d = pd.to_datetime("2022-11-01")
'-'.join(str(x).zfill(2) for x in [d.year, d.month, d.day])

In [ ]:
submissions_df['row_id'] = submissions_df['cfips'] + '_' + submissions_df['date'].apply(lambda d: '-'.join(str(x).zfill(2) for x in [d.year, d.month, d.day]))
submissions_df

In [ ]:
submissions_df[['row_id', 'microbusiness_density']].to_csv("submission-naive-01.csv", index=False)

## Naive baseline #2
* AR(1)